# Protrein generative models evaluation metrics
## Environment Preparation
### Conda Environment

You can set up the conda environment by running the following command:

In [ ]:
conda env create -f eval.yml

Besides, you have to install the following packages:

``` bash

pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
pip install fair-esm


```

### Foldseek database

When we calculate the novelty metric, we use the Foldseek database.


``` bash

conda install -c conda-forge -c bioconda foldseek
mkdir ./foldseek_database
cd ./foldseek_database
foldseek databases PDB pdb tmp 

```

Foldseek will download the PDB database automatically. After the download, you directory should look like this:

```
foldseek_database
    ├── pdb
    ├── pdb_ca
    ├── pdb_ca.dbtype
    ├── pdb_ca.index
    ├── pdb_clu
    ├── pdb_clu.dbtype
    ├── pdb_clu.index
    ├── pdb.dbtype
    ├── pdb_h
    ├── pdb_h.dbtype
    ├── pdb_h.index
    ├── pdb.index
    ├── pdb.lookup
    ├── pdb_mapping
    ├── pdb_seq.0 -> pdb
    ├── pdb_seq.1
    ...
```

After downloading the foldseek database, you need to replace the database path in the `foldseek_database` field of the `configs/evaluation.yaml` file.

### Maxcluster

When we cluster the designed protein based on their structure, we use maxcluster to cluster them.

``` bash
wget https://www.sbg.bio.ic.ac.uk/maxcluster/maxcluster64bit
```

#### Example data

We provide some example data `./example_data` for testing purposes.

```
└── length_70
    ├── sample_0
    │   ├── bb_traj.pdb
    │   ├── sample.pdb
    │   └── x0_traj.pdb
    ├── sample_1
    │   ├── bb_traj.pdb
    │   ├── sample.pdb
    │   └── x0_traj.pdb
```



### ProteinMPNN

We can use the ProteinMPNN model to design a sequence for a given structure. 

``` bash

git clone https://github.com/dauparas/ProteinMPNN.git

```

## Evaluation

### Single pdb evaluation

In [1]:
# import package
import os
import time
import numpy as np
import hydra
import torch
import subprocess
import logging
import pandas as pd
import shutil
from datetime import datetime
from biotite.sequence.io import fasta
import GPUtil
from typing import Optional, Union, List
from analysis import utils as au
from analysis import metrics
from data import utils as du
from omegaconf import DictConfig, OmegaConf
from openfold.data import data_transforms
import esm
from pathlib import Path
import mdtraj as md
from openfold.np import residue_constants
from tmtools import tm_align
from openfold.utils.superimposition import superimpose
from tqdm import tqdm
import re
import yaml

In [ ]:
class DictToObject:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            if isinstance(value, dict):
                setattr(self, key, DictToObject(value))
            else:
                setattr(self, key, value)

def load_config(file_path):
    with open(file_path, "r") as f:
        config = yaml.safe_load(f)
    return config


In [3]:
from EvalRunner import EvalRunner
config_dict = load_config("./configs/evaluation.yaml")
conf = DictToObject(config_dict)
EvalModel = EvalRunner(conf)


#### Self consistency metrics

```mermaid

graph TD;
    A[Protein Generative models] --> B[ProteinMPNN (inverse folding)];
    B --> C[ESMFold (folding)];

```


In [5]:
# Example pdb path
pdb_path = "/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_0/"
sc_output_dir = os.path.join(pdb_path, "self_consistency")
os.makedirs(sc_output_dir, exist_ok=True)

sc_results = EvalModel.calc_designability(sc_output_dir, pdb_path)
sc_results

,tm_score,bb_rmsd,sample_path,header,sequence
0,0.285942,12.353213,/home/shuaikes/server2/shuaikes/projects/prote...,"sample, score=2.0021, global_score=2.0021, fix...",AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
1,0.820439,2.127891,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=1, score=1.0243, global_score=1....",ATLTKMLVKVKDKSITLEDVKKIIKEVGVDAEIEIDKETNTVTITA...
2,0.838250,2.160685,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=2, score=1.0038, global_score=1....",SILTKLKIKIKDKSINLEDIKKIAKEEGINCKIEIDKETNEVIVEA...
3,0.852542,1.917273,/home/shuaikes/server2/shuaikes/projects/prote...,"T=0.1, sample=3, score=0.9787, global_score=0....",MVKTKMKIKIKDKSINKEDIEKIVKEEGLNVEIEIDKDTNTVTVKG...


#### Sub-structure ratio evaluation

In [6]:
path = os.path.join(pdb_path, "sample.pdb")
sub_ratio = EvalModel.calc_mdtraj_metrics(path)
sub_ratio

{'non_coil_percent': 0.6571428571428571,
 'coil_percent': 0.34285714285714286,
 'helix_percent': 0.34285714285714286,
 'strand_percent': 0.3142857142857143,
 'radius_of_gyration': 1.0974538862859071}

#### Novelty: pdbTM

In [ ]:
path = os.path.join(pdb_path, "sample.pdb")
value = EvalModel.pdbTM(pdb_path)
value

TM-Score between sample.pdb and its closest protein in PDB is 0.867.


#### Calculate all metrics

In [9]:
EvalModel.calc_all_metrics(sc_output_dir, pdb_path)

Calculating all metrics...
##########################################################################
Calculating designability...
   tm_score    bb_rmsd                                        sample_path  \
0  0.285942  12.353213  /home/shuaikes/server2/shuaikes/projects/prote...   
1  0.820439   2.127891  /home/shuaikes/server2/shuaikes/projects/prote...   
2  0.838250   2.160685  /home/shuaikes/server2/shuaikes/projects/prote...   
3  0.852542   1.917273  /home/shuaikes/server2/shuaikes/projects/prote...   

                                              header  \
0  sample, score=2.0021, global_score=2.0021, fix...   
1  T=0.1, sample=1, score=1.0243, global_score=1....   
2  T=0.1, sample=2, score=1.0038, global_score=1....   
3  T=0.1, sample=3, score=0.9787, global_score=0....   

                                            sequence  
0  AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...  
1  ATLTKMLVKVKDKSITLEDVKKIIKEVGVDAEIEIDKETNTVTITA...  
2  SILTKLKIKIKDKSINLEDIKKIAKEEGINCKIE

#### Diversity: number of clusters

We are not able to calculate the diversity of the cluster for just single protein strucuture. So we need multiple pdb files here and we use csv to specify the path the protein structures.

Your csv file should like this:

```
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_0/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_1/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_2/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_3/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_4/
/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/example_data/length_70/sample_5/
```

In [ ]:
pdb_csv_path = "/home/shuaikes/server2/shuaikes/projects/protein-evaluation-notebook/pdb_path.csv"
clusters = EvalModel.calc_diversity(pdb_csv_path)
clusters

6
